## Python imports

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from jinja2 import Template
import os
import sys

### _Load Django into Jupyter_

In [ ]:
sys.path.append('/bookstore')
sys.path.append('/bookstore/project')
os.environ['DJANGO_SETTINGS_MODULE'] = 'project.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'
import django
django.setup()

### _Obtain App and DB models_

In [ ]:
from django.apps import apps
app_models = [v for k,v in apps.all_models['bookstore'].items()]

dbdesign = [{'model': model._meta.object_name, \
             'fields': [field.name for field in \
                        model._meta.get_fields()]} \
            for model in app_models]

### Add DRF Token Authentication

In [ ]:
old = "WSGI_APPLICATION = 'project.wsgi.application'\n\n\n"
new = "WSGI_APPLICATION = 'project.wsgi.application'\n\
\n\
REST_FRAMEWORK = {\n\
    'DEFAULT_FILTER_BACKENDS': [\n\
        'django_filters.rest_framework.DjangoFilterBackend'\n\
    ],\n\
    'DEFAULT_AUTHENTICATION_CLASSES': [\n\
        'rest_framework.authentication.BasicAuthentication',\n\
        'rest_framework.authentication.SessionAuthentication',\n\
        'rest_framework_simplejwt.authentication.JWTAuthentication'\n\
    ],\n\
}\n\n\n"

with open("/bookstore/project/settings.py", "r+") as file:
    data = file.read()
    data = data.replace(old, new)
    with open("/bookstore/project/settings.py", "w") as file:
        write_file = file.write(data)

## Adding Views

In [ ]:
from django.apps import apps
file = '/bookstore/bookstore/views.py'  
    
with open(file, 'w') as file:
    template = Template(open('/templates/views.j2', 'r').read())
    print(template.render(dbdesign=[app['model'] for app in dbdesign]))
    file.write(template.render(dbdesign=[app['model'] for app in dbdesign]))

## Adding URLs

In [ ]:
from django.apps import apps
file = '/bookstore/bookstore/urls.py' 
    
with open(file, 'w') as file:
    template = Template(open('/templates/urls_app.j2', 'r').read())
    print(template.render(dbdesign=[app['model'] for app in dbdesign]))
    file.write(template.render(dbdesign=[app['model'] for app in dbdesign]))

In [ ]:
from django.apps import apps
file = '/bookstore/project/urls.py'   
appname = [app for app in apps.app_configs][-1]

with open(file, 'w') as file:
    template = Template(open('/templates/urls.j2', 'r').read())
    print(template.render(app_name=appname))
    file.write(template.render(app_name=appname))

### _let's check . . ._

In [ ]:
url = 'http://bookstore:8000/api'

def page_title(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

page_title(url)

![title](images/drf.png)

## JSON browsing

In [ ]:
url = 'http://bookstore:8000/api'
headers = {'Content-type': 'application/json'}
requests.get(url, headers=headers).json()

In [ ]:
url = "http://bookstore:8000/api"
payload = {'username': 'carlos.carrot', 'password': 'secret123'}
token = requests.post(f'{url}/token/', json=payload, headers=headers).json()['access']
headers['Authorization'] = f"Bearer {token}"

print(json.dumps(headers, indent=4, sort_keys=True))

In [ ]:
url = "http://bookstore:8000/api"
for model in [app['model'] for app in dbdesign if 'Historical' not in app['model']]:
    response = requests.get(f"{url}/{model.lower()}", headers=headers).json()
    print(f"A total of {len(response)} {model} API Objects")
#     print(json.dumps(response[-2:], indent=4, sort_keys=True))